In [1]:
%load_ext blackcellmagic

In [2]:
import pandas as pd
import gurobipy as grb
import gurobipy_pandas as gpd

### Data Sets

* Let $\mathcal T$ be the set of tasks that need to be completed.

### Data Inputs

* Let $\eta_t \ge 0$ be the number of hours for a task $t\in\mathcal T$.

In [3]:
tasks = pd.read_excel("../data/data.xlsx", 
                      sheet_name="Tasks", 
                      index_col=0)
display(tasks)

,Time
Task,
1,4
2,3
3,1
4,3
5,3
6,3
7,5
8,1
9,2


### Data Sets

* Let $\mathcal G$ be the set of resource groups.

### Data Inputs

* Let $\alpha_g \ge 0$ be the minimum number of tasks, $t\in\mathcal T$, that a resource group $g\in\mathcal G$ can work in one day.
* Let $\beta_g \ge 0$ be the maximum number of tasks, $t\in\mathcal T$, that a resource group $g\in\mathcal G$ can work in one day.


In [4]:
resource_groups = pd.read_excel("../data/data.xlsx", sheet_name='Resource Groups', index_col=0)
resource_groups

,MinTasks,MaxTasks
ResourceGroup,,
Tiger,3,9
Lion,3,13
Cougar,3,3


### Data Sets

* Let $\mathcal R$ be the set of resources that can complete the tasks.
* Let $\mathcal S_g$ be the set of resources that belong to resource group $g \in\mathcal G$.

### Data Inputs

* Let $\lambda_r \ge 0$ be the cost per hour for a resource $r\in\mathcal R$.
* Let $\gamma_r \ge 0$ be the number of hours that a resource $r\in\mathcal R$ can work in one day.

In [5]:
resources = pd.read_excel("../data/data.xlsx", 
                          sheet_name='Resources',
                          index_col=0)
resources

,HoursAvailable,CostPerHour,ResourceGroup
Resource,,,
A,9,21,Cougar
B,15,22,Tiger
C,13,29,Lion
D,14,28,Tiger
E,13,24,Lion
F,7,18,Lion


### Data Sets

* Let $\mathcal Q_r$ be the set of tasks that resource $r\in\mathcal R$ can perform.
* Let $\mathcal P_t$ be the set of resources that a task $t\in\mathcal T$ can be assigned to.

In [6]:
res_task = pd.read_excel("../data/data.xlsx", sheet_name='Tasks For Resources', index_col=[1,0]).drop(columns='CanPerform').sort_values("Task")
res_task

Empty DataFrame
Columns: []
Index: [(1, A), (1, E), (1, D), (1, C), (1, B), (2, F), (2, E), (2, D), (3, E), (4, C), (4, B), (4, D), (4, A), (5, B), (6, F), (6, A), (7, F), (7, D), (8, B), (8, E), (8, A), (8, F), (8, D), (9, F), (9, B), (9, E), (10, A), (10, D), (10, C), (11, E), (11, F), (11, D), (12, D), (12, B), (12, E), (12, A), (12, C), (13, F), (13, D), (13, C), (14, F), (14, D), (14, E), (14, A), (14, B), (15, D), (15, C), (15, F), (15, B), (16, E), (16, C), (16, A), (16, D), (16, F), (17, A), (17, D), (17, E), (17, C), (17, B), (18, C), (18, A), (18, D), (19, A), (19, D), (19, B), (19, E), (20, A), (20, F)]

[68 rows x 0 columns]

## Data Validation

In [7]:
assert (
    tasks["Time"].max() < resources["HoursAvailable"].max()
), f"The max task time {tasks['Time'].max()} exceeds the maximum resource availability {resources['HoursAvailable'].max()}"

In [8]:
P = res_task.groupby('Task').agg(list)
assert len(tasks) <= len(
    P
), f"Not all tasks have resources that can schedule them: "f"{', '.join([str(t) for t in set(tasks.index)- set(P.index)])}"

In [9]:
assert (
    tasks["Time"].sum() < resources["HoursAvailable"].sum()
), f"Total Task Time: {tasks['Time'].sum()} Total Resource Time: {resources['HoursAvailable'].sum()}"

### Data Transformation

* Let $\theta_{rt}$ be the cost for a resource $r\in\mathcal R$ performing a task $t\in\mathcal T$.

$$ \theta_{rt} = \eta_t \cdot \lambda_r \qquad\forall r\in\mathcal R,\,t\in\mathcal T$$

In [10]:
(res_task.merge(resources[["CostPerHour"]], 
           how="left", left_on="Resource", 
           right_index=True)
    .merge(tasks[["Time"]], how="left", 
           left_on="Task", right_index=True)
    .assign(theta=lambda df: df["CostPerHour"] * df["Time"]))

CostPerHour  Time  theta
Task Resource                          
1    A                  21     4     84
     E                  24     4     96
     D                  28     4    112
     C                  29     4    116
     B                  22     4     88
...                    ...   ...    ...
19   D                  28     1     28
     B                  22     1     22
     E                  24     1     24
20   A                  21     4     84
     F                  18     4     72

[68 rows x 3 columns]

In [11]:
theta = (
    res_task
    .merge(resources[["CostPerHour"]], 
           how="left", left_on="Resource", 
           right_index=True)
    .merge(tasks[["Time"]], how="left", 
           left_on="Task", right_index=True)
    .assign(theta=lambda df: df["CostPerHour"] * df["Time"])["theta"]
)
theta.to_frame("theta").sort_index()

theta
Task Resource       
1    A            84
     B            88
     C           116
     D           112
     E            96
...              ...
19   B            22
     D            28
     E            24
20   A            84
     F            72

[68 rows x 1 columns]

In [12]:
m = grb.Model('task_assign')

Set parameter TokenServer to value "gurobi.princeton.com"


### Decision Variables

* Let $x_{rt}\in\{0, 1\}$ be one if the resource $r\in\mathcal R$ is assigned to perform task $t\in\mathcal T$, 0 otherwise.
* Let $y_g\in \mathbb{Z} \ge 0$ be the number of tasks assigned to resources in $g\in\mathcal G$.

In [13]:
x = gpd.add_vars(m, res_task.index, name="x", vtype=grb.GRB.BINARY)
m.update()
x.to_frame('x').query("Task in [4,5]")

x
Task Resource                     
4    C         <gurobi.Var x[4,C]>
     B         <gurobi.Var x[4,B]>
     D         <gurobi.Var x[4,D]>
     A         <gurobi.Var x[4,A]>
5    B         <gurobi.Var x[5,B]>

In [14]:
y = resource_groups.gppd.add_vars(m, vtype=grb.GRB.INTEGER, name="y")['y']
m.update()
y.to_frame("y")

,y
ResourceGroup,
Tiger,<gurobi.Var y[Tiger]>
Lion,<gurobi.Var y[Lion]>
Cougar,<gurobi.Var y[Cougar]>


In [15]:
from typing import Any, List, Tuple, Union


def namer(name: str, ind_val: Union[Tuple, List[Any], str, int]) -> str:
    """Function to name a variable or constraint based on the index
     values and the given base name.

    Args:
        name (str): Base name for variable or constraint
        ind_val (Union[Tuple, List[Any], str, int]): Index for the variable or
         constraint that makes it unique amongst the set for this base name.

    Returns:
        str: Unique name for this variable or constraint.
    """
    if isinstance(ind_val, str) or isinstance(ind_val, int):
        ind_vals = str(ind_val)
    else:
        ind_vals = ",".join([str(i) for i in list(ind_val)])
    ret_val = f"{name}[{ind_vals}]"
    return ret_val

### Assign Each Task to only One Resource

Each task can only be assigned to be performed one task in the model.

$$\sum_{r\in\mathcal R} x_{rt} = 1 \qquad\forall t\in\mathcal T$$

In [16]:
df = gpd.add_constrs(m, x.groupby('Task').agg(grb.quicksum), grb.GRB.EQUAL, 1, name='AllTasksMustBeScheduled')
m.update()
df

Task
1      <gurobi.Constr AllTasksMustBeScheduled[1]>
2      <gurobi.Constr AllTasksMustBeScheduled[2]>
3      <gurobi.Constr AllTasksMustBeScheduled[3]>
4      <gurobi.Constr AllTasksMustBeScheduled[4]>
5      <gurobi.Constr AllTasksMustBeScheduled[5]>
6      <gurobi.Constr AllTasksMustBeScheduled[6]>
7      <gurobi.Constr AllTasksMustBeScheduled[7]>
8      <gurobi.Constr AllTasksMustBeScheduled[8]>
9      <gurobi.Constr AllTasksMustBeScheduled[9]>
10    <gurobi.Constr AllTasksMustBeScheduled[10]>
11    <gurobi.Constr AllTasksMustBeScheduled[11]>
12    <gurobi.Constr AllTasksMustBeScheduled[12]>
13    <gurobi.Constr AllTasksMustBeScheduled[13]>
14    <gurobi.Constr AllTasksMustBeScheduled[14]>
15    <gurobi.Constr AllTasksMustBeScheduled[15]>
16    <gurobi.Constr AllTasksMustBeScheduled[16]>
17    <gurobi.Constr AllTasksMustBeScheduled[17]>
18    <gurobi.Constr AllTasksMustBeScheduled[18]>
19    <gurobi.Constr AllTasksMustBeScheduled[19]>
20    <gurobi.Constr AllTasksMustBeScheduled[

### Ensure Each Resource Stays within their Allowed Hours

Each resource can only be assigned tasks whose total time must be no more than their maximum hours for a day.

$$\sum_{t\in\mathcal T} \eta_t \cdot x_{rt} \le \gamma_r \qquad\forall r\in\mathcal R$$

In [17]:
df_res_ub = pd.concat(
    [
        pd.merge(x, tasks["Time"], how="left", left_on="Task", right_index=True)
        .assign(eta_x=lambda df: df["Time"] * df["x"])
        .groupby(["Resource"])["eta_x"]
        .agg(grb.quicksum),
        resources["HoursAvailable"],
    ],
    axis=1,
).gppd.add_constrs(m,"eta_x", grb.GRB.LESS_EQUAL, 'HoursAvailable', name='ResourceCantExceedHours')
m.update()
df_res_ub

,eta_x,HoursAvailable,ResourceCantExceedHours
Resource,,,
A,"<gurobi.LinExpr: 4.0 x[1,A] + 3.0 x[4,A] + 3.0...",9,<gurobi.Constr ResourceCantExceedHours[A]>
B,"<gurobi.LinExpr: 4.0 x[1,B] + 3.0 x[4,B] + 3.0...",15,<gurobi.Constr ResourceCantExceedHours[B]>
C,"<gurobi.LinExpr: 4.0 x[1,C] + 3.0 x[4,C] + 3.0...",13,<gurobi.Constr ResourceCantExceedHours[C]>
D,"<gurobi.LinExpr: 4.0 x[1,D] + 3.0 x[2,D] + 3.0...",14,<gurobi.Constr ResourceCantExceedHours[D]>
E,"<gurobi.LinExpr: 4.0 x[1,E] + 3.0 x[2,E] + x[3...",13,<gurobi.Constr ResourceCantExceedHours[E]>
F,"<gurobi.LinExpr: 3.0 x[2,F] + 3.0 x[6,F] + 5.0...",7,<gurobi.Constr ResourceCantExceedHours[F]>


### Count Tasks Assigned to Resource Group

Determine the number of tasks assigned to resources in each resource group

$$\sum_{r\in\mathcal S_g}\sum_{t\in\mathcal Q_r} x_{rt} = y_g \qquad\forall g \in\mathcal G$$

In [18]:
df_res_group_set_tasks = pd.concat(
	[
		pd.merge(x, resources["ResourceGroup"],
				how="left",
				left_on="Resource",
				right_index=True,
				)
			.groupby("ResourceGroup")["x"]
			.agg(sum_x=grb.quicksum),
		y,
	], axis=1).gppd.add_constrs(m, "sum_x", grb.GRB.EQUAL, "y", name="CountTasksForGroup")
m.update()
df_res_group_set_tasks


,sum_x,y,CountTasksForGroup
ResourceGroup,,,
Cougar,"<gurobi.LinExpr: x[1,A] + x[4,A] + x[6,A] + x[...",<gurobi.Var y[Cougar]>,<gurobi.Constr CountTasksForGroup[Cougar]>
Lion,"<gurobi.LinExpr: x[1,E] + x[1,C] + x[2,F] + x[...",<gurobi.Var y[Lion]>,<gurobi.Constr CountTasksForGroup[Lion]>
Tiger,"<gurobi.LinExpr: x[1,D] + x[1,B] + x[2,D] + x[...",<gurobi.Var y[Tiger]>,<gurobi.Constr CountTasksForGroup[Tiger]>


### Resource Groups are Task Count Limited

#### Ensure Resource Group Meets Minimum Task Processing

Ensure each resource group is only scheduled up to its maximum number of tasks

$$y_g \ge \alpha_g \qquad\forall g\in\mathcal G$$

#### Ensure Resource Group Meets Maximum Task Processing

Ensure each resource group is only scheduled up to its maximum number of tasks

$$y_g \le \beta_g \qquad\forall g\in\mathcal G$$

In [19]:
df = (
    pd.concat([y, resource_groups], axis=1)
    .gppd.add_constrs(
        m, "y", grb.GRB.GREATER_EQUAL, "MinTasks", name="GroupMeetsMinTasks"
    )
    .gppd.add_constrs(m, 'y', grb.GRB.LESS_EQUAL, "MaxTasks", name="GroupMeetsMaxTasks")
)
m.update()
df

,y,MinTasks,MaxTasks,GroupMeetsMinTasks,GroupMeetsMaxTasks
ResourceGroup,,,,,
Tiger,<gurobi.Var y[Tiger]>,3,9,<gurobi.Constr GroupMeetsMinTasks[Tiger]>,<gurobi.Constr GroupMeetsMaxTasks[Tiger]>
Lion,<gurobi.Var y[Lion]>,3,13,<gurobi.Constr GroupMeetsMinTasks[Lion]>,<gurobi.Constr GroupMeetsMaxTasks[Lion]>
Cougar,<gurobi.Var y[Cougar]>,3,3,<gurobi.Constr GroupMeetsMinTasks[Cougar]>,<gurobi.Constr GroupMeetsMaxTasks[Cougar]>


### Objective

The objective function for the model is to minimize the cost of assigning the resources to tasks.

$$\text{Minimize}\,\sum_{r\in\mathcal R}\sum_{t\in\mathcal T} \theta_{rt}\cdot x_{rt}$$

In [20]:
z = grb.quicksum(theta * x)
m.setObjective(z, grb.GRB.MINIMIZE)
m.update()
z

<gurobi.LinExpr: 84.0 x[1,A] + 96.0 x[1,E] + 112.0 x[1,D] + 116.0 x[1,C] + 88.0 x[1,B] + 54.0 x[2,F] + 72.0 x[2,E] + 84.0 x[2,D] + 24.0 x[3,E] + 87.0 x[4,C] + 66.0 x[4,B] + 84.0 x[4,D] + 63.0 x[4,A] + 66.0 x[5,B] + 54.0 x[6,F] + 63.0 x[6,A] + 90.0 x[7,F] + 140.0 x[7,D] + 22.0 x[8,B] + 24.0 x[8,E] + 21.0 x[8,A] + 18.0 x[8,F] + 28.0 x[8,D] + 36.0 x[9,F] + 44.0 x[9,B] + 48.0 x[9,E] + 63.0 x[10,A] + 84.0 x[10,D] + 87.0 x[10,C] + 120.0 x[11,E] + 90.0 x[11,F] + 140.0 x[11,D] + 56.0 x[12,D] + 44.0 x[12,B] + 48.0 x[12,E] + 42.0 x[12,A] + 58.0 x[12,C] + 90.0 x[13,F] + 140.0 x[13,D] + 145.0 x[13,C] + 36.0 x[14,F] + 56.0 x[14,D] + 48.0 x[14,E] + 42.0 x[14,A] + 44.0 x[14,B] + 56.0 x[15,D] + 58.0 x[15,C] + 36.0 x[15,F] + 44.0 x[15,B] + 72.0 x[16,E] + 87.0 x[16,C] + 63.0 x[16,A] + 84.0 x[16,D] + 54.0 x[16,F] + 84.0 x[17,A] + 112.0 x[17,D] + 96.0 x[17,E] + 116.0 x[17,C] + 88.0 x[17,B] + 145.0 x[18,C] + 105.0 x[18,A] + 140.0 x[18,D] + 21.0 x[19,A] + 28.0 x[19,D] + 22.0 x[19,B] + 24.0 x[19,E] + 84.0 x[

In [21]:
m.write('grbpd.lp')

In [22]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 35 rows, 71 columns and 213 nonzeros
Model fingerprint: 0x4f99366c
Variable types: 0 continuous, 71 integer (68 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 1507.0000000
Presolve removed 11 rows and 6 columns
Presolve time: 0.00s
Presolved: 24 rows, 65 columns, 192 nonzeros
Variable types: 0 continuous, 65 integer (63 binary)
Found heuristic solution: objective 1471.0000000

Root relaxation: objective 1.436000e+03, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1436.00000    0    8 1471.00000 1436.00000  2.38% 

In [23]:
m.write('model.lp')

In [24]:
assignment_data = pd.DataFrame(index=x.index).assign(IsAssigned=[v.X for v in x])
assignment_data = (
    assignment_data.query("IsAssigned > 0.1").astype(int)
)
assignment_data = assignment_data.merge(
    tasks[["Time"]], how="left", left_on="Task", right_index=True
)
assignment_data

,,IsAssigned,Time
Task,Resource,,
1,B,1,4
2,E,1,3
3,E,1,1
4,A,1,3
5,B,1,3
6,F,1,3
7,D,1,5
8,A,1,1
9,B,1,2


In [25]:
import plotly.graph_objects as go

In [26]:
fig = go.Figure()
res = list(resources.index)

for it in assignment_data.reset_index().sort_values('Task', ascending=False).itertuples():
    y_dict = {r:0 for r in res}
    y_dict[it.Resource] = it.Time
    fig.add_trace(go.Bar(x=res, y=list(y_dict.values()), name=it.Task))

    
fig.add_trace(go.Scatter(x=res, y=resources.HoursAvailable.values, name='Max Hours', showlegend=False))
fig.update_layout(barmode='stack', title='Task Assignments', xaxis={'title':"Resources"}, 
                  legend={'title':"Task"}, yaxis={'title': "Time (Hours)"}, width=800, autosize=False)
fig.add_annotation(
            x='A',
            y=resources.query('Resource=="A"').HoursAvailable.values[0],
            text="Max Hours for Resource")
fig

In [27]:
assignment_data.reset_index().sort_values('Task', ascending=False)

,Task,Resource,IsAssigned,Time
19,20,F,1,4
18,19,E,1,1
17,18,A,1,5
16,17,D,1,4
15,16,E,1,3
14,15,B,1,2
13,14,B,1,2
12,13,D,1,5
11,12,B,1,2
10,11,E,1,5


In [28]:
resource_group_assignment = pd.concat([resource_groups, 
pd.concat([assignment_data.groupby('Resource').size().to_frame("NumTasks"), resources[['ResourceGroup']]], axis=1).groupby("ResourceGroup").sum()], axis=1)
resource_group_assignment

,MinTasks,MaxTasks,NumTasks
ResourceGroup,,,
Tiger,3,9,9
Lion,3,13,8
Cougar,3,3,3


In [29]:
fig = go.Figure()
res_groups = sorted(resource_groups.index)
res_assignments = pd.concat([assignment_data.groupby('Resource').size().to_frame("NumTasks"), 
                             resources[['ResourceGroup']]], axis=1).reset_index().set_index(["Resource", "ResourceGroup"]).fillna(0).astype(int).reset_index()

for r in sorted(res_assignments.Resource.values, reverse=True):
    group_tasks = pd.concat([resource_groups, res_assignments.set_index(['ResourceGroup', 'Resource']).xs(r, level="Resource")], axis=1).fillna(0)['NumTasks']
    fig.add_trace(go.Bar(name=r, 
                         x=res_groups, 
                         y=[group_tasks.loc[rg] for rg in res_groups],
                         legendgroup='resource',
                         legendgrouptitle_text="Resource",))
fig.add_trace(go.Scatter(x=res_groups, y=[resource_groups.MaxTasks.loc[rg] for rg in res_groups], name='Max', legendgroup='tasks', legendgrouptitle_text='Task Count Bounds'))
fig.add_trace(go.Scatter(x=res_groups, y=[resource_groups.MinTasks.loc[rg] for rg in res_groups], name='Min', legendgroup='tasks'))
fig.update_layout(barmode='stack', title="Resource Group Usage", yaxis={'title':'Num Tasks Assigned'}, xaxis={'title':'Resource Group'}, autosize=False, height=500, width=1000)
fig.add_annotation(
            x='Lion',
            y=resource_groups.query('ResourceGroup=="Lion"').MaxTasks.values[0],
            text="Max Tasks for Resource Group")
fig.add_annotation(
            x='Tiger',
            y=resource_groups.query('ResourceGroup=="Tiger"').MinTasks.values[0],
            text="Min Tasks for Resource Group")